## RDP - Stone Soup #5: Data Association with Clutter 
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #5: Data Association with clutter.

For this tutorial, I need to extend the test data set that I've been working with to include some clutter. I'll grab all plots within the timeframe of the test target that were not correlated to the test plane or any other targets of opportunity. 

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all, CSVClutterReaderXY


from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection, Clutter
from stonesoup.plotter import AnimatedPlotterly, Plotter

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

plot_type = 'static' # or 'animated'

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


In [ ]:
target_address = 10537421
# adsb_sql = f"""SELECT `timestamp`,
#         time_of_day, 
#         latitude, 
#         longitude, 
#         target_address,
#         flight_level, 
#         rho, 
#         theta
# FROM radar_data.adsb
# WHERE test_date = '2024-07-17'
# AND target_address={target_address}
# and latitude is not NULL
# AND rho<20
# ORDER BY `timestamp`"""

# adsb_straight = dfunc.query_to_df(adsb_sql)

# rdp_sql = f"""SELECT 
#         `timestamp`,
#         time_of_day,
#         cal, 
#         rho,
#         theta, 
#         x, 
#         y, 
#         field_note 
# FROM radar_data.rdp

# --Clutter Where Statement
# WHERE test_date = '2024-07-17'
# AND `timestamp` >= '{start_time.strftime("%Y-%m-%d %H:%M:%S")}'
# AND `timestamp` <= '{end_time.strftime("%Y-%m-%d %H:%M:%S")}'
# AND sortie_id=61
# ORDER BY `timestamp`"""

# # # Test Plane Where
# # # WHERE `timestamp` >= '{adsb_straight.timestamp.min().strftime("%Y-%m-%d %H:%M:%S")}'
# # # AND `timestamp` <= '{adsb_straight.timestamp.max().strftime("%Y-%m-%d %H:%M:%S")}'
# # # AND rho >= {adsb_straight.rho.min()-0.5}
# # # AND rho <= {adsb_straight.rho.max()+0.5}
# # # AND theta >= {adsb_straight.theta.min()- 5}
# # # AND theta <= {adsb_straight.theta.max()+5}"""

# # # rdp_straight = dfunc.query_to_df(rdp_sql)
# # # rdp_straight.head()

# rdp_clutter = dfunc.query_to_df(rdp_sql)
# rdp_clutter.head()

### Gather Clutter Data

In [ ]:
# # remove timezone from timestamps
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].dt.tz_localize(None)
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].astype('datetime64[us]')
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].round('ms')

# all_matched_data['timestamp'] = all_matched_data['timestamp_rdp'].dt.tz_localize(None)
# all_matched_data['timestamp'] = all_matched_data['timestamp'].astype('datetime64[us]')
# all_matched_data['timestamp'] = all_matched_data['timestamp'].round('ms')

# # # Select only needed columns
# cols_to_keep = ['timestamp','rho','theta','cal','x','y']
# rdp_clutter = rdp_clutter[cols_to_keep]

# # Left JOIN with the matched data to eliminate all RDP plots matched to any targets of opportunity
# rdp_clutter = pd.merge(rdp_clutter, all_matched_data.loc[all_matched_data.close_enough==1], 
#                                left_on=['rho','theta','timestamp'], 
#                                right_on = ['rho_rdp','theta_rdp','timestamp'], 
#                                how='left')

# # Drop rows that matched a target of opportunity
# rdp_clutter = rdp_clutter.loc[rdp_clutter.rho_rdp.isna()]
# # filter for only the range where the test plane is to simplify things
# rdp_clutter = rdp_clutter.loc[(rdp_clutter.rho>=matched_data.rho_rdp.min()) &
#                               (rdp_clutter.rho<=matched_data.rho_rdp.max())&
#                               (rdp_clutter.theta>=matched_data.theta_rdp.min()) &
#                               (rdp_clutter.theta<=matched_data.theta_rdp.max()) & 
#                               (rdp_clutter['timestamp'] >= matched_data.timestamp_rdp.min()) &
#                               (rdp_clutter['timestamp'] <= matched_data.timestamp_rdp.max())]

# rdp_clutter['theta_rad'] = np.deg2rad(rdp_clutter.theta)

# data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
# clutter_filename = f'{data_dir}/sample_clutter.csv'
# rdp_clutter.to_csv(clutter_filename, index=False)
# len(rdp_clutter)

## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
# adsb_straight.to_csv(adsb_file, index=False)
adsb_data = pd.read_csv(adsb_file)
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)

# Matched Plots
matched_csv = f'{data_dir}/rdp_matched.csv'
rdp_matched = pd.read_csv(matched_csv)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()


clutter_filename = f'{data_dir}/sample_clutter.csv'
clutter_data = pd.read_csv(clutter_filename)

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

In [ ]:
v.scatter_targets(clutter_data)
plt.suptitle('Clutter')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
all_matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = all_matched_data.loc[(all_matched_data.target_address==target_address) &
                                (all_matched_data.close_enough==True)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
rdp_matched = matched_data[['timestamp_rdp',
                            'cal_rdp',
                            'rho_rdp',
                            'theta_rdp', 
                            'target_address']]

rdp_matched['theta_rad'] = np.deg2rad(rdp_matched.theta_rdp)
rdp_matched.rename(columns={'rho_rdp': 'rho',
                            'theta_rdp': 'theta', 
                            'timestamp_rdp': 'timestamp', 
                            'cal_rdp': 'cal'}, 
                            inplace=True)

rdp_matched['x'], rdp_matched['y'] = zip(*rdp_matched.apply(lambda x: dfunc.polar_to_cartesian(x.rho, x.theta), axis=1))

matched_csv = f'{data_dir}/rdp_matched.csv'
rdp_matched.to_csv(matched_csv, index=False)

rdp_matched.head()

In [ ]:
rdp_matched['x_m'] = rdp_matched.x * METERS_in_NM
rdp_matched['y_m'] = rdp_matched.y * METERS_in_NM
rdp_matched.sort_values('timestamp', inplace=True)

track_start_x = rdp_matched.iloc[0]['x_m']
track_start_y = rdp_matched.iloc[0]['y_m']

rdp_matched[['timestamp', 'rho','theta','x_m','y_m']].head()

In [ ]:
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

# Detections
matched_xy = CSVReaderXY(matched_csv)
matched_polar = CSVReaderPolar(matched_csv)

# Cutter
clutter = CSVClutterReaderXY(clutter_filename)

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

plot_type='static'
# plot_type='animated'
timestamps = generate_timestamps(start_time, end_time)

# plot_all(all_measurements, adsb, start_time, end_time, plot_type='static')
plot_all(all_measurements, adsb, start_time, end_time, plot_type='animated')


## From Tutorial #5

### Probability of detection
For the first time we introduce the possibility that, at any time-step, our sensor receives no detection from the target (i.e. $p_d < 1$).

In [ ]:
prob_det = 0.9

## Simulate clutter
Next in the tutorial, they simulate clutter. Using LWR data, we have plenty of clutter to work with. For this example, I've taken all of the RDP data within the same timeframe, range and azimuth ranges for the test plane and removed all plots associated with any targets of opportunity, including the test plane. Looking at the plot of remaining "clutter," it is clear that some of those plots belong to a track. Those aircraft were either:
1. not squawking its position and therefore missing from our 'truth' data
2. not captured by the ADS-B recorder, again missing from the 'truth'
2. not correctly correlated to the track 
3. correlated to the track, but outside of the designated range and azimuth tolerances

## Distance Hypothesiser and Nearest Neighbour
Perhaps the simplest way to associate a detection with a prediction is to measure a 'distance' to each detection and hypothesise that the detection with the lowest distance
is correctly associated with that prediction.

An appropriate distance metric for states described by Gaussian distributions is the *Mahalanobis distance*. This quantifies the distance of a point relative to a given distribution. In the case of a point $\mathbf{x} = [x_{1}, ..., x_{N}]^T$, and distribution with mean $\boldsymbol{\mu} = [\mu_{1}, ..., \mu_{N}]^T$ and covariance matrix $P$, the Mahalanobis distance of $\mathbf{x}$ from the distribution is given by:

$$
       \sqrt{(\mathbf{x} - \boldsymbol{\mu})^T P^{-1} (\mathbf{x} - \boldsymbol{\mu})}
$$
which equates to the multi-dimensional measure of how many standard deviations a point is away from the mean.

We're going to create a hypothesiser that ranks detections against predicted measurement according to the Mahalanobis distance, where those that fall outside of :math:`3` standard deviations of the predicted measurement's mean are ignored. To do this we create a `DistanceHypothesiser` which pairs incoming detections with track predictions, and pass it a `Measure` class which (in this instance) calculates the Mahalanobis distance.

The hypothesiser must use a predicted state given by the predictor, create a measurement prediction using the updater, and compare this to a detection given a specific metric. Hence, it takes the predictor, updater, measure (metric) and missed distance (gate) as its arguments. We therefore need to create a predictor and updater, and to initialise a measure.

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

from stonesoup.types.track import Track
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis, Euclidean

from stonesoup.dataassociator.neighbour import NearestNeighbour

Now we use the `NearestNeighbour` data associator, which picks the hypothesis pair (predicted measurement and detection) with the highest 'score' (in this instance, those that are closest to each other).

 <div>
 <img src="https://stonesoup.readthedocs.io/en/latest/_images/NN_Association_Diagram.png" width="500"/>
 <div>

 In the diagram above, there are three possible detections to be considered for association (some of which may be clutter). The detection with a score of `0.4` is selected by the nearest neighbour algorithm.

## Run the Kalman filter with the associator
With these components, we can run the simulated data and clutter through the Kalman filter. However, the process is expecting the data in sets by second where the true detections and clutter are sets grouped by second. We need to reshape our data to match that format for the data associater to work properly.

***Note:*** The per-second nature of the sample might not be the best way to evaluate real data. Per-scan, or per-sector may make more sense. On a per-scan basis, we'd expect to see each active target in every scan. If looking by sector, we'd only expect to see each track updated every 32 sectors, or thereabouts, assuming the radar will not see a target twice in one time around.

In [ ]:
from datetime import datetime
from collections import defaultdict

# Group objects by the second of their timestamp
grouped_objects = defaultdict(set)

for meas in all_measurements:
# for meas in dets:
    # Get the second part of the timestamp
    timestamp_s = meas.timestamp.replace(microsecond=0)  # Truncate to second precision
    grouped_objects[timestamp_s].add(meas)
    # print(timestamp_s)

# Convert grouped objects to a set of sets
grouped_measurements = set(frozenset(group) for group in sorted(grouped_objects.values()))

In [ ]:
grouped_list = [group for group in grouped_objects.values()]
# grouped_list

In [ ]:
# Measurement Model
np.random.seed(42)

default_variance = 50 # estimate of variance in m2 of state matrix elements (position and velocity)

measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[default_variance, 0 ],  
                          [0, default_variance]]),
    seed=24
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 40
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)],
                                                          seed=24)

In [ ]:
# not sure why, but running a second time works much better. Something must be getting calibrated the first time
# and lingering to make the second track better. Need to identify that something and, if possible, set it as an
# initial condition.
for i in range (0,1): 
    # Create prior
    predictor = KalmanPredictor(transition_model)
    updater = KalmanUpdater(measurement_model)

    # predictor = UnscentedKalmanPredictor(transition_model)
    # updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

    hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)
    # hypothesiser = DistanceHypothesiser(predictor, updater, measure=Euclidean(), missed_distance=80)
    data_associator = NearestNeighbour(hypothesiser)

    # Clear things out from prior runs
    hypothesis = None
    post = None
    if "track" in globals():
        del(track)

    # create a prior using the approximate start of the track
    prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

    # create a prior using the location of the radar
    # prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

    # Loop through the predict, hypothesise, associate and update steps.

    track = Track([prior])
    for n, measurements in enumerate(grouped_list):
        this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
        this_time = this_time.replace(microsecond=0)
        # print(f'{this_time}: {len(measurements)}')

        if len(measurements)>0:
        # for n, measurements in enumerate(dets):
            try: 
                hypotheses = data_associator.associate([track],
                                                    measurements,
                                                    this_time)
                hypothesis = hypotheses[track]
            
                if hypothesis.measurement:
                    post = updater.update(hypothesis)
                    track.append(post)
                else:  # When data associator says no detections are good enough, we'll keep the prediction
                    track.append(hypothesis.prediction)

                # print(f'{this_time}: {len(measurements)}: SUCCESS')

            except:
                # print(f'{this_time}: {len(measurements)}: ERROR')
                continue

## Plot the ground truth and measurements with clutter.

In [ ]:
start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()
meas_cart = CSVClutterReaderXY(matched_csv)
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

plot_all(all_measurements, adsb, start_time, end_time, tracks=[track], plot_type='static')

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

### All Measurements - from Polar
Recreate the all_measurements collection using the polar version.

In [ ]:
dets = [next(iter(detection[1])) for detection in matched_polar.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

# Group objects by the second of their timestamp
grouped_objects = defaultdict(set)

for meas in all_measurements:
# for meas in dets:
    # Get the second part of the timestamp
    timestamp_s = meas.timestamp.replace(microsecond=0)  # Truncate to second precision
    grouped_objects[timestamp_s].add(meas)
    # print(timestamp_s)

# Convert grouped objects to a set of sets
grouped_measurements = set(frozenset(group) for group in sorted(grouped_objects.values()))

len(grouped_measurements)

### Kalman Filtering Again...
Should be the same from here forward.

In [ ]:
# Transition Model
# q_const = 50
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

# Create prior
# predictor = KalmanPredictor(transition_model)
# updater = KalmanUpdater(measurement_model)

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

# Clear things out from prior runs
hypothesis = None
post = None
if "track" in globals():
    del(track)

data_associator = NearestNeighbour(hypothesiser)

# create a prior using the approximate start of the track
prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
track = Track([prior])
for n, measurements in enumerate(grouped_list):
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)
    # print(f'{this_time}: {len(measurements)}')

    if len(measurements)>0:
        # print(f'{n}: {len(measurements)}')
    # for n, measurements in enumerate(dets):
        try: 
            hypotheses = data_associator.associate([track],
                                                   measurements,
                                                   this_time)
            hypothesis = hypotheses[track]
        
            if hypothesis.measurement:
                post = updater.update(hypothesis)
                track.append(post)
            else:  # When data associator says no detections are good enough, we'll keep the prediction
                track.append(hypothesis.prediction)

            # print(f'{this_time}: {len(measurements)}: SUCCESS')

        except:
            # print(f'{this_time}: {len(measurements)}: ERROR')
            continue

len(track)

In [ ]:
plot_all(all_measurements, adsb, start_time, end_time, tracks=track, plot_type='static')

The results are very sensitive to the constant velocity parameter. They are also changing when re-running without making changes! I suspect the Kalman Filter is creating covariance matrices or something that are not getting reset when re-run.

* 25: falls short of the ground truth consistently in the y- coordinate.
* 35: does a pretty good job of matching the truth.
* 50: matches well up until a point, then veers off track northerly for no apparent reason at